# Exploration

In [1]:
import pandas as pd
import csv
import re
from collections import Counter
from itertools import islice
from tqdm import tqdm_notebook
from fog.key import fingerprint

In [2]:
# Params
DATA = '../data/interventions.csv'
OUTPUT = '../data/didascalies.csv'

# Match for "group"
GROUPS = [
    ('LREM', ('larem', 'lrem', 'rem'), 'en marche'),
    ('MODEM', ('modem', ), 'mouvement démocrate'),
    ('LR', ('lr', ), 'républicains'),
    ('LFI', ('lfi', 'fi'), 'insoumise'),
    ('LC-UDI-UAI', ('udi', 'lc', 'uai'), 'constructifs'),
    ('NG', ('ng', ), 'nouvelle gauche'),
    ('GDR', ('gdr', ), 'gauche démocrate'),
    ('NI', ('ni', ), 'non inscrit')
]

GROUPS = [(t, tuple(re.compile('\\b' + p + '\\b') for p in ps), e) for t, ps, e in GROUPS]

BLACK_LIST = [
    'article',
    'motion',
    'séance', # Possibilité d'étudier les suspensions via ce mot-clé
    'amendement',
    'projet',
    'scrutin'
]

EXCLAMATION_RE = re.compile('«([^»]+)»')
HYPHENS_RE = re.compile('[\\-‐‒–—―−‑⁃]')
QUOTES_RE = re.compile('[«»]')
DIDASCALIES_SPLITTER_RE = re.compile('[.\s]-')

In [3]:
# Helpers
TAGS_RE = re.compile(r'<[^>]+>')

def strip_tags(html):
    return re.sub(TAGS_RE, '', html)

def is_didascalie(line):
    if line['nom'] != 'NULL' or line['parlementaire'] != 'NULL':
        return False
    
    raw_intervention = fingerprint(strip_tags(re.sub(QUOTES_RE, '', line['intervention'])))
    
    if fingerprint(re.sub(QUOTES_RE, '', line['titre'])) == raw_intervention:
        return False
    
    for b in BLACK_LIST:
        if b in raw_intervention:
            return False
    
    return True

def split_didascalies(intervention):
    intervention = strip_tags(intervention)
    intervention = re.sub(HYPHENS_RE, '-', intervention)
    
    return [d.strip('. ') for d in DIDASCALIES_SPLITTER_RE.split(intervention.strip('. '))]

def is_relevant(intervention):
    intervention = intervention.lower()
    
    return (
        'applaud' in intervention or
        'murmur' in intervention or
        'rire' in intervention or
        'banc' in intervention
    )

def extract_groups(didascalie):
    raw_didascalie = didascalie.lower()
    
    groups = set()
    
    for name, patterns, expression in GROUPS:
        if expression in raw_didascalie:
            groups.add(name)
            
            continue
            
        for pattern in patterns:
            if re.search(pattern, raw_didascalie):
                groups.add(name)
                break
                
    return list(groups)

In [4]:
def find_reference_point(lines, didascalie_line, index):
    index -= 1
    
    while is_didascalie(lines[index]):
        index -=1
        
    if lines[index]['seance_id'] != didascalie_line['seance_id']:
        print(didascalie_line)
        print(lines[index])
        print()
        
    return lines[index]

In [5]:
DIDASCALIES = []

with open(DATA, 'r') as f:
    reader = csv.DictReader(f)
    
    LINES = list(reader)
    
    for i, line in enumerate(LINES):
        if not is_didascalie(line):
            continue
            
        didascalies = split_didascalies(line['intervention'])
        
        reference_point = find_reference_point(LINES, line, i)
        
        for didascalie in didascalies:
            meta = dict(reference_point)
            meta['didascalie'] = didascalie

            DIDASCALIES.append(meta)

In [6]:
Counter(d['didascalie'].lower() for d in DIDASCALIES if 'même' in d['didascalie'].lower()).most_common()

[('mêmes mouvements', 485),
 ('applaudissements sur les mêmes bancs', 14),
 ('mêmes mouvements sur les mêmes bancs', 10),
 ('même mouvement', 8),
 ('nouveaux applaudissements sur les mêmes bancs', 3),
 ('exclamations persistantes sur les mêmes bancs', 3),
 ('même mouvements', 3),
 ('exclamations sur les mêmes bancs', 2),
 ('protestations sur les mêmes bancs', 2),
 ('mêmes mouvement', 2),
 ('« eh oui ! » sur les mêmes bancs', 1),
 ('« si ! » sur les mêmes bancs', 1),
 ('nouvelles protestations sur les mêmes bancs', 1),
 ('« non ! » sur les mêmes bancs', 1),
 ('brouhaha grandissant sur les mêmes bancs', 1),
 ('nouvelles exclamations sur les mêmes bancs', 1),
 ('mêmes mouvements et exclamations sur les bancs des groupes rem et modem',
  1),
 ('« ah ! » sur les mêmes bancs', 1),
 ('vifs applaudissements sur les mêmes bancs', 1),
 ('« bravo ! » et rires sur les mêmes bancs', 1),
 ('les mêmes se lèvent et applaudissent longuement', 1),
 ('« pas de la même manière ! »sur les bancs du groupe l

In [7]:
# Handling "mêmes mouvements"
MEMES_MOUVEMENTS_RE = re.compile('mêmes?\s+mouvements?')

for i, d in enumerate(DIDASCALIES):
    if re.search(MEMES_MOUVEMENTS_RE, d['didascalie'].lower()):
        # print(d['didascalie'])
        last_d = DIDASCALIES[i - 1]
        meta = dict(last_d)
        meta['didascalie'] = d['didascalie']
        DIDASCALIES[i] = meta

In [8]:
# Extracting groups
for d in DIDASCALIES:
    groups = extract_groups(d['didascalie'])
    d['groups'] = '|'.join(groups)

In [9]:
# All groups at once
ALL_GROUPS_KEY = '|'.join(g for g, _, _ in GROUPS)
for d in DIDASCALIES:
    raw_didascalie = d['didascalie'].lower()
        
    if 'tous les banc' in raw_didascalie or 'mmes et mm. les députés' in raw_didascalie:
        d['groups'] = ALL_GROUPS_KEY

In [10]:
# Handling "mêmes bancs"
MEMES_BANCS_RE = re.compile('mêmes?\s+bancs?')

for i, d in enumerate(DIDASCALIES):
    raw_didascalie = d['didascalie'].lower()
    
    if re.search(MEMES_BANCS_RE, raw_didascalie):
        last_d = DIDASCALIES[i - 1]
        
        groups = set(d['groups'].split('|') if d['groups'] else []) | set(last_d['groups'].split('|') if last_d['groups'] else [])

        d['groups'] = '|'.join(groups)

In [11]:
pd.DataFrame(DIDASCALIES).head(1000)

,date,didascalie,fonction,groups,id,intervention,moment,nb_mots,nom,parlementaire,parlementaire_groupe_acronyme,seance_id,sexe,source,timestamp,titre,titre_complet,type
0,2017-06-27,La séance est ouverte à quinze heures,"président, doyen d'âge",,1,<p>La séance est ouverte.</p>,15:00,7,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,20,NULL,,loi
1,2017-06-27,Applaudissements,"président, doyen d'âge",,14,<p>Monsieur le secrétaire d'État chargé des re...,15:00,636,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,170,allocution du doyen d'âge,allocution du doyen d'âge,loi
2,2017-06-27,Il est procédé au tirage au sort,"président, doyen d'âge",,17,"<p>L'ordre du jour appelle, conformément à l'a...",15:00,140,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,220,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
3,2017-06-27,Il est procédé au tirage au sort,"président, doyen d'âge",,19,<p>Sont désignés scrutateurs titulaires : M. B...,15:00,51,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,250,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
4,2017-06-27,"La séance, suspendue à seize heures cinq, est ...","président, doyen d'âge",,24,<p>La séance est suspendue.</p>,15:00,7,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,320,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
5,2017-06-27,Mmes et MM. les députés et M. le secrétaire d'...,"président, doyen d'âge",LREM|MODEM|LR|LFI|LC-UDI-UAI|NG|GDR|NI,26,<p>La séance est reprise.</p><p>Mes chers coll...,15:00,63,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,350,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
6,2017-06-27,Applaudissements,"président, doyen d'âge",,28,<p>M. Jean-Charles Taugourdeau : 94 voix</p>,15:00,6,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,370,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
7,2017-06-27,Applaudissements,"président, doyen d'âge",,30,<p>Mme Laure de La Raudière : 34 voix</p>,15:00,9,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,390,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
8,2017-06-27,Applaudissements,"président, doyen d'âge",,32,<p>Mme Laurence Dumont : 32 voix</p>,15:00,6,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,410,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
9,2017-06-27,Applaudissements,"président, doyen d'âge",,34,<p>Mme Caroline Fiat : 30 voix</p>,15:00,6,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,430,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi


In [12]:
Counter(d['parlementaire'] if d['parlementaire'] != 'NULL' else d['nom'] for d in DIDASCALIES if 'applaud' in d['didascalie'].lower()).most_common()

[('Edouard Philippe', 348),
 ('Gérard Collomb', 305),
 ('Bruno le Maire', 282),
 ('Gérald Darmanin', 277),
 ('NULL', 229),
 ('Agnès Buzyn', 226),
 ('Jean-Luc Mélenchon', 223),
 ('Éric Coquerel', 210),
 ('Muriel Pénicaud', 195),
 ('Danièle Obono', 184),
 ('André Chassaigne', 158),
 ('Sébastien Jumel', 154),
 ('Nicole Belloubet', 151),
 ('François Ruffin', 148),
 ('Alexis Corbière', 136),
 ('François de Rugy', 125),
 ('Julien Denormandie', 125),
 ('Stéphane Travert', 123),
 ('Jean-Michel Blanquer', 122),
 ('Jacques Mézard', 120),
 ('Ugo Bernalicis', 117),
 ('Stéphane Peu', 117),
 ("Loïc Prud'homme", 110),
 ('Bastien Lachaud', 110),
 ('Marc Fesneau', 107),
 ('Gilles Lurton', 106),
 ('Fabien Di Filippo', 103),
 ('Nicolas Hulot', 102),
 ('Christian Jacob', 97),
 ('Jacqueline Gourault', 93),
 ('élisabeth Borne', 93),
 ('Adrien Quatennens', 91),
 ('Mathilde Panot', 91),
 ('Clémentine Autain', 90),
 ('Christophe Castaner', 90),
 ('Fabien Roussel', 89),
 ('Jean-Paul Lecoq', 89),
 ('Richard Ferr

In [13]:
Counter(d['parlementaire_groupe_acronyme'] for d in DIDASCALIES if 'applaud' in d['didascalie'].lower()).most_common()

[('NULL', 3560),
 ('LREM', 3485),
 ('LR', 2274),
 ('LFI', 1803),
 ('GDR', 1030),
 ('MODEM', 929),
 ('NG', 863),
 ('UAI', 472),
 ('LC', 289),
 ('NI', 279)]

In [14]:
Counter(d['sexe'] for d in DIDASCALIES if 'applaud' in d['didascalie'].lower()).most_common()

[('H', 7872), ('NULL', 3560), ('F', 3552)]

In [15]:
Counter(d['didascalie'] for d in DIDASCALIES if d['parlementaire'] == 'Jean Lassalle').most_common()

[('Sourires', 33),
 ('Rires', 13),
 ('Applaudissements sur divers bancs', 3),
 ('Applaudissements sur les bancs des groupes FI et GDR', 2),
 ('Applaudissements sur les bancs du groupe LR', 2),
 ('rires sur plusieurs bancs', 1),
 ('Applaudissements et rires sur plusieurs bancs', 1),
 ('Applaudissements sur les bancs des groupes REM et MODEM, ainsi que sur plusieurs bancs de chacun des autres groupes',
  1),
 ('Applaudissements sur plusieurs bancs de chacun des groupes', 1),
 ('Sourires sur plusieurs bancs', 1),
 ("Nombre de votants263Nombre de suffrages exprimés258Majorité absolue130Pour l'adoption86contre172",
  1),
 ('Exclamations', 1),
 ('Plusieurs députés du groupe LR se lèvent et applaudissent', 1),
 ('Applaudissements sur les bancs du groupe REM et sur quelques bancs du groupe FI',
  1),
 ('Applaudissements sur plusieurs bancs du groupe REM', 1),
 ('Murmures', 1),
 ('Rires et applaudissements sur de nombreux bancs', 1),
 ('Applaudissements sur les bancs du groupe LR et sur quelque

In [16]:
for d in DIDASCALIES:
    raw_didascalie = d['didascalie'].lower()
    
    if 'pupitr' in raw_didascalie or 'claquem' in raw_didascalie:
        print(d['didascalie'])

Claquements de pupitre sur un banc du groupe LR
Claquements de pupitres sur plusieurs bancs des groupes REM et MODEM
Exclamations et claquements de pupitres sur les bancs du groupe REM
Claquements de pupitre sur quelques bancs du groupe REM
Claquements de pupitres sur les bancs du groupe REM
Exclamations et claquements de pupitre sur les bancs du groupe REM
Très vives protestations et claquements de pupitres sur les bancs des groupes LR, UDI-Agir, NG et GDR
Claquements de pupitres sur les bancs du groupe LaREM
Exclamations et claquements de pupitres sur les bancs du groupe LR
Claquements de pupitres sur certains bancs des députés non inscrits
Exclamations sur les bancs des groupes FI et NG, claquements de pupitre sur les bancs du groupe GDR
Claquements de pupitres sur les bancs du groupe GDR, puis des groupes LR, FI et NG
Les claquements de pupitres s'intensifient
Les claquements de pupitres sur les autres bancs couvrent la voix de Mme la secrétaire d'État
De nombreux députés du groupe

In [17]:
with open(OUTPUT, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=list(DIDASCALIES[0].keys()))
    writer.writeheader()
    
    for d in DIDASCALIES:
        writer.writerow(d)